In [ ]:
import tifffile as tiff
from natsort import natsorted
import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import cv2
from imutils.src import imfunctions #contours_length as imutils.contours_length
from natsort import natsorted
import shutil

In [ ]:
#Code to make binary

In [ ]:

#this script seems to be doing pretty much what make_binary was doing (maybe better and maybe more)
#Subsample a stack series
#reader and writer objects
input_filepath='/groups/zimmer/Ulises/wbfm/chemotaxis_assay/2020_Only_behaviour/btf_all/2020-07-01_14-41-11_chemotaxisl_worm2-channel-0-bigtiff.btf'
image_output_filepath='/groups/zimmer/Ulises/wbfm/chemotaxis_assay/2020_Only_behaviour/2020-07-01_14-41-11_chemotaxisl_worm2-channel-0-bigtiff_image_subsampled.btf'
mask_output_filepath='/groups/zimmer/Ulises/wbfm/chemotaxis_assay/2020_Only_behaviour/2020-07-01_14-41-11_chemotaxisl_worm2-channel-0-bigtiff_mask_subsampled.btf'
bg_img_filename='/groups/zimmer/Ulises/wbfm/chemotaxis_assay/2020_Only_behaviour/background/2020-07-01_13-42-05_chemotaxisl_worm1_bg-channel-0-_MMStack_Median_BG.tiff'
bg_img= tiff.imread(bg_img_filename)

expected_worm_size=7000
#output_tif=tiff.TiffWriter(image_output_filepath, bigtiff=True)

output_image_writer=tiff.TiffWriter(image_output_filepath, bigtiff=True)
output_mask_writer=tiff.TiffWriter(mask_output_filepath, bigtiff=True)


#with tiff.TiffWriter(output_filepath, bigtiff=True) as output_tif:
with tiff.TiffFile(input_filepath, multifile=True) as tif:
    #for our data there would be only one series
    for i, page in enumerate(tif.pages):
        #if i<22000: continue
        if i%5000!=0:continue #use nly every 50 image
        #if i>24100: break
        img=page.asarray()
        img=cv2.bitwise_not(img)
        img=cv2.subtract(img,bg_img)


        output_image_writer.write(img, photometric='minisblack',contiguous=True)

        #median Blur
        img=cv2.medianBlur(img,5)

        #apply threshold
        ret, new_img = cv2.threshold(img,20,255,cv2.THRESH_BINARY)

        #contours
        _,cnts,hierarchy = cv2.findContours(new_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        #print('cnts is : ',type(cnts))
        #print(cnts)
        cnt_areas=[]
        cnts_idx=[]#np.array([])
        img_contours=np.zeros(img.shape)

        for cnt_idx, cnt in enumerate(cnts):
            cnt_area=cv2.contourArea(cnt)
            if (expected_worm_size*0.8 <cnt_area<expected_worm_size*1.4):
                cnt_areas.append(cnt_area)
                cnts_idx.append(np.array(cnt_idx))
                #print('area', cnt_area, 'cnt_idx', cnt_idx)
                cv2.drawContours(img_contours,cnts, cnt_idx, color=255, thickness=-1, hierarchy=hierarchy, maxLevel=1)

            if hierarchy[0][cnt_idx][3] in cnts_idx:
                if cnt_area<300:#cv2.contourArea(contours[j])<300:
            #print(cv2.contourArea(contours[j]))
            #print('entered')
                    cv2.drawContours(img_contours,cnts, cnt_idx, color=255, thickness=-1)

        mask=img*img_contours
        img_contours=np.array(img_contours, dtype=np.uint8)
        #plotting
#         fig, (ax1,ax2,ax3)=plt.subplots(figsize=(10,5), nrows=1, ncols=3, dpi=200)
#         im=ax1.imshow(img)
#         divider = make_axes_locatable(ax1)
#         cax = divider.append_axes('right', size='5%', pad=0.05)
#         fig.colorbar(im, cax=cax, orientation='vertical')
#         ax2.imshow(img_contours)            
#         ax3.imshow(mask)#, vmin=20, vmax=255)
#         #plt.show()
#         [axi.set_axis_off() for axi in (ax1,ax2,ax3)]
#         fig.suptitle([i, cnts_idx], fontsize=25)

        #saving new_img
        output_mask_writer.write(img_contours, photometric='minisblack',contiguous=True)
        #if i >20000: break
output_image_writer.close()
output_mask_writer.close()

In [ ]:
image_path='/groups/zimmer/Ulises/wbfm/chemotaxis_assay/2020_Only_behaviour/2020-07-01_14-41-11_chemotaxisl_worm2-channel-0-bigtiff_image_subsampled.btf'
#mask_path='/groups/zimmer/Ulises/code/unet-master/data/training/2020-07-01_14-41-11_chemotaxisl_worm2-channel-0-bigtiff_image_subsampled.tif'


stack2images(image_path,'/groups/zimmer/Ulises/code/unet-master/data/training/image')
#imfunctions.stack2images(mask_path,'/groups/zimmer/Ulises/code/unet-master/data/training/mask')

In [ ]:
#First part
frames2extract=np.arange(0,200000,10000)
output_folder='/groups/zimmer/Ulises/code/unet-master/data/new_training/'
input_folder='/groups/zimmer/Ulises/wbfm/chemotaxis_assay/2020_Only_behaviour/btf_all/'
for btf_file in natsorted(os.listdir(input_folder)):
    print(btf_file)
    if '2020-07' in btf_file:
        print(btf_file)
        btf_file_basename=os.path.splitext(os.path.basename(btf_file))[0]
        print(btf_file_basename)
        imfunctions.extract_frames(os.path.join(input_folder,btf_file), os.path.join(output_folder,btf_file_basename),frames2extract)
        print('end')

In [ ]:
#remove background
#add zeros and rename if necessary before this step

bg_img_filename='/groups/zimmer/Ulises/wbfm/chemotaxis_assay/2020_Only_behaviour/background/2020-07-01_13-42-05_chemotaxisl_worm1_bg-channel-0-_MMStack_Median_BG.tiff'
bg_img=tiff.imread(bg_img_filename)

path='/groups/zimmer/Ulises/code/unet-master/data/new_training_test'

for directory, subdirectories, files in os.walk(path):
    for file in files:
        filepath=os.path.join(directory,file)
        img=tiff.imread(filepath)
        inv_img=cv2.bitwise_not(img)
        new_img=cv2.subtract(inv_img,bg_img)
        tiff.imwrite(filepath,new_img)

In [ ]:
#draw_contours
path='/groups/zimmer/Ulises/code/unet-master/data/new_training_test'

for directory, subdirectories, files in os.walk(path):
    for file in files:
        filepath=os.path.join(directory,file)
        img=tiff.imread(filepath)
        #median Blur
        img=cv2.medianBlur(img,5)
       
        #apply threshold
        ret, new_img = cv2.threshold(img,20,255,cv2.THRESH_BINARY)
        worm_contour_img=imfunctions.draw_some_contours(new_img,contour_size=7000,tolerance=0.2, inner_contour_area_to_fill=300)
        tiff.imwrite(filepath,worm_contour_img)

In [ ]:
#add zeros
path='/groups/zimmer/Ulises/code/unet-master/data/new_training_test'
for dirpath, _, files in os.walk(path):
    print(dirpath)
    imfunctions.add_zeros_to_filename(dirpath, 6)

In [ ]:
#rename to have unique names (prepend folder name into file name)
path='/groups/zimmer/Ulises/code/unet-master/data/new_training_test'
for dirpath, _, files in os.walk(path):
    for f in files:
        os.rename(os.path.join(dirpath, f), os.path.join(dirpath, os.path.split(dirpath)[-1] + '_' + f))

In [ ]:
#copy all files from subfolders into another folder
path='/groups/zimmer/Ulises/code/unet-master/data/new_training_test'
dst_path='/groups/zimmer/Ulises/code/unet-master/data/new_training_test2'

try: os.mkdir(dst_path) # creates the subdirectory where it should be stored
except: print('Output Directory already exists, might overwrite')

for directory, subdirectories, files in os.walk(path):
    for file in files:
        #print(file)
        shutil.copyfile(os.path.join(directory,file),os.path.join(dst_path,file))
